In [ ]:
import csv
import pandas as pd
import requests

In [ ]:
def update_csv_file(csv_file_path, *database_lists):
    """
    Combine multiple lists of databases, remove duplicates based on URL, and write the results to a CSV file.

    Args:
        csv_file_path (str): Path to the CSV file to be created/updated.
        *database_lists (list of dicts): Variable number of lists containing database information.
    """
    # Combine all provided database lists into one, using the URL as a unique identifier to remove duplicates
    combined_databases = {db['URL']: db for database_list in database_lists for db in database_list}.values()
    
    # Write the combined list to the CSV file
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=[
            "Paper Title", "Received Date", "Published Date", "Name of Journal",
            "First Author", "First Author Affiliation", "First Author Email", 
            "Second Author", "Second Author Affiliation", "Second Author Email",
            "Third Author", "Third Author Affiliation", "Third Author Email",
            "Corresponding Author", "Corresponding Author Email",
            "volume", "Number", "Article Number", "DOI", "Abstract"
            ])
        writer.writeheader()
        for db in combined_databases:
            writer.writerow(db)

Use template below to create new row in dataset.

Use format below to update above table. Use following prompt to search journals using ChatGPT:

List all peer reviewed paper from elsivier published in last 50 years in the format below. Exclude [] in the code.
{
    "Paper Title": "",
    "Received Date": " ",
    "Published Date": " ",
    "Name of Journal": " ",
    "First Author": " ",
    "First Author Affiliation":, " "
    "First Author Email": " ", 
    "Second Author": " ",
    "Second Author Affiliation": " ",
    "Second Author Email": " ",
    "Third Author": " ",
    "Third Author Affiliation": " ",
    "Third Author Email": " ",
    "KeyWord1": " ",
    "KeyWord2": " ",
    "KeyWord3": " ",
    "KeyWord4": " ",
    "KeyWord5": " ",
    "KeyWord6": " ",
    "Volume": " ",
    "Number": " ",
    "Article Number": " ",
    "DOI": " ",
    "Abstract": " "
    }

In [ ]:
your_scopus_api_key = "f67ff601f6de466a22ca3c340044d507"
api_key = 'f67ff601f6de466a22ca3c340044d507'

In [ ]:
# Replace with your new or verified API key
api_key = "your_scopus_api_key"
base_url = "https://api.elsevier.com/content/search/scopus"

headers = {
    'X-ELS-APIKey': api_key,
    'Accept': 'application/json'
}

# Initialize the papers list
papers = []

# Search query for peer-reviewed papers from Elsevier in the last 10 years
params = {
    'query': 'PUBYEAR > 2014',
    'count': 25,
    'sort': 'date',
    'start': 0
}

response = requests.get(base_url, headers=headers, params=params)

# Checking the response status and handling errors
if response.status_code == 200:
    data = response.json()
    entries = data.get('search-results', {}).get('entry', [])

    for entry in entries:
        # Extracting author details with safer access
        authors = entry.get('author', [])
        affiliations = entry.get('affiliation', [])
        authkeywords = entry.get('authkeywords', [])

        # Check for corresponding author in the response
        corresponding_author = entry.get(
            'corresponding-author', {}) if 'corresponding-author' in entry else {}

        # Safe access to list elements
        def safe_get(lst, index, default_value):
            try:
                return lst[index]
            except IndexError:
                return default_value

        paper = {
            "Paper Title": entry.get('dc:title', ''),
            "Received Date": entry.get('prism:coverDisplayDate', ''),
            "Published Date": entry.get('prism:coverDate', ''),
            "Name of Journal": entry.get('prism:publicationName', ''),
            "First Author": safe_get(authors, 0, {}).get('authname', ''),
            "First Author Affiliation": safe_get(affiliations, 0, {}).get('affilname', ''),
            "First Author Email": safe_get(authors, 0, {}).get('author-email', ''),
            "Second Author": safe_get(authors, 1, {}).get('authname', ''),
            "Second Author Affiliation": safe_get(affiliations, 1, {}).get('affilname', ''),
            "Second Author Email": safe_get(authors, 1, {}).get('author-email', ''),
            "Third Author": safe_get(authors, 2, {}).get('authname', ''),
            "Third Author Affiliation": safe_get(affiliations, 2, {}).get('affilname', ''),
            "Third Author Email": safe_get(authors, 2, {}).get('author-email', ''),
            "Corresponding Author": corresponding_author.get('authname', ''),
            "Corresponding Author Email": corresponding_author.get('author-email', ''),
            "KeyWord1": safe_get(authkeywords, 0, ''),
            "KeyWord2": safe_get(authkeywords, 1, ''),
            "KeyWord3": safe_get(authkeywords, 2, ''),
            "KeyWord4": safe_get(authkeywords, 3, ''),
            "KeyWord5": safe_get(authkeywords, 4, ''),
            "KeyWord6": safe_get(authkeywords, 5, ''),
            "Volume": entry.get('prism:volume', ''),
            "Number": entry.get('prism:issueIdentifier', ''),
            "Article Number": entry.get('dc:identifier', ''),
            "DOI": entry.get('prism:doi', ''),
            "Abstract": entry.get('dc:description', '')
        }
        papers.append(paper)

    # Display the collected papers
    for paper in papers:
        print(paper)

else:
    print("Failed to retrieve data:", response.status_code, response.text)

In [ ]:
# Filepath to save the CSV
csv_file_path = "Elsivier Papers.csv"

# Call the function with any number of database lists
update_csv_file(csv_file_path, bibs)

print(f"CSV file '{csv_file_path}' has been updated.")

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Get the dimensions of the DataFrame
num_rows, num_columns = df.shape

print(f"The CSV file has {num_rows} rows and {num_columns} columns.")